In [2]:
#imports
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn import model_selection, metrics
from imblearn.over_sampling import SMOTE
import csv

In [3]:
# Util functions
def classifierEstimation(classifier, x_test, y_test):
    print
    print('Test set has ', len([i for i in y_test if i is 1]), 'positives and ', len([i for i in y_test if i is 0]), ' negatives')
    print
    print ('Best classifier score')
    print
    print(metrics.classification_report(y_test, classifier.predict(x_test), target_names=['non-binding', 'binding']))

In [4]:
#import data

positives_dna = []
negatives_dna = []
positives_enzyme = []
negatives_enzyme = []

with open('data/minimal_model_sw_pdidb_positives.txt') as p_dna:
    for row in csv.reader(p_dna,delimiter="\t"):
        positives_dna.append(row)

with open('data/minimal_model_sw_pdidb_negatives.txt') as n_dna:
    for row in csv.reader(n_dna,delimiter="\t"):
        negatives_dna.append(row)
       
with open('data/minimal_model_sw_enzyme_positives.txt') as p_enzyme:
    for row in csv.reader(p_enzyme,delimiter="\t"):
        positives_enzyme.append(row)
        
with open('data/minimal_model_sw_enzyme_negatives.txt') as n_enzyme:
    for row in csv.reader(n_enzyme,delimiter="\t"):
        negatives_enzyme.append(row)

In [5]:
#import random data

positives_dna_rnd = []
negatives_dna_rnd = []
positives_enzyme_rnd = []
negatives_enzyme_rnd = []

with open('data/rnd_more_minimal_model_pdidb_positives.txt') as p_dna:
    for row in csv.reader(p_dna,delimiter="\t"):
        positives_dna_rnd.append(row)

with open('data/rnd_more_minimal_model_pdidb_negatives.txt') as n_dna:
    for row in csv.reader(n_dna,delimiter="\t"):
        negatives_dna_rnd.append(row)
       
with open('data/rnd_more_minimal_model_enzyme_positives.txt') as p_enzyme:
    for row in csv.reader(p_enzyme,delimiter="\t"):
        positives_enzyme_rnd.append(row)
        
with open('data/rnd_more_minimal_model_enzyme_negatives.txt') as n_enzyme:
    for row in csv.reader(n_enzyme,delimiter="\t"):
        negatives_enzyme_rnd.append(row)
        
tmp_dna_rnd = positives_dna_rnd + negatives_dna_rnd
tmp_enzyme_rnd = positives_enzyme_rnd + negatives_enzyme_rnd

labels_dna_rnd = [1]*len(positives_dna_rnd)+[0]*len(negatives_dna_rnd)
labels_enzyme_rnd = [1]*len(positives_enzyme_rnd)+[0]*len(negatives_enzyme_rnd)

dna_rnd = []
enzyme_rnd = []

for j in tmp_dna_rnd:
    new_j = [float(i) for i in j]
    dna_rnd.append(new_j)

for j in tmp_enzyme_rnd:
    new_j = [float(i) for i in j]
    enzyme_rnd.append(new_j)

data_rnd = dna_rnd + enzyme_rnd
labels_rnd = labels_dna_rnd + labels_enzyme_rnd

dna_x_train_rnd,dna_x_test_rnd,dna_y_train_rnd,dna_y_test_rnd = model_selection.train_test_split(dna_rnd,labels_dna_rnd,test_size=0.2,random_state=6)
enzyme_x_train_rnd,enzyme_x_test_rnd,enzyme_y_train_rnd,enzyme_y_test_rnd = model_selection.train_test_split(enzyme_rnd,labels_enzyme_rnd,test_size=0.2,random_state=6)

x_test_rnd = dna_x_test_rnd + enzyme_x_test_rnd
y_test_rnd = dna_y_test_rnd + enzyme_y_test_rnd

In [6]:
#preprocess data

tmp_dna = positives_dna + negatives_dna
tmp_enzyme = positives_enzyme + negatives_enzyme

labels_dna = [1]*len(positives_dna)+[0]*len(negatives_dna)
labels_enzyme = [1]*len(positives_enzyme)+[0]*len(negatives_enzyme)

dna = []
enzyme = []

for j in tmp_dna:
    new_j = [float(i) for i in j]
    dna.append(new_j)
    
for j in tmp_enzyme:
    new_j = [float(i) for i in j]
    enzyme.append(new_j)

In [7]:
#train and test datasets

dna_x_train,dna_x_test,dna_y_train,dna_y_test = model_selection.train_test_split(dna, labels_dna, test_size=0.2, random_state=6)
enzyme_x_train,enzyme_x_test,enzyme_y_train,enzyme_y_test = model_selection.train_test_split(enzyme,labels_enzyme,test_size=0.2,random_state=6)

x_train = dna_x_train + enzyme_x_train
y_train = dna_y_train + enzyme_y_train
x_test = dna_x_test + enzyme_x_test
y_test = dna_y_test + enzyme_y_test

sm = SMOTE(random_state=42)
x_train, y_train = sm.fit_sample(x_train,y_train)

print(x_train)
print(x_test)

kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=3)
parameters = {'alpha': [0.0001], 'random_state': [1]}

D:\Program Files\Python\Python3.6\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: setting an array element with a sequence.

In [8]:
#training the neural network

grid = model_selection.GridSearchCV(MLPClassifier(), parameters, cv=kf)
grid.fit(x_train,y_train)
classifier = grid.best_estimator_

NameError: name 'parameters' is not defined

In [51]:
#testing

#prediction = classifier.predict(enzyme_x_test)
prediction = classifier.predict(x_test)
unique, counts = np.unique(prediction, return_counts=True)
print(dict(zip(unique, counts)))

classifierEstimation(classifier,enzyme_x_test,enzyme_y_test)
classifierEstimation(classifier,dna_x_test, dna_y_test)
classifierEstimation(classifier,x_test,y_test)

print("Training set score: %f" % classifier.score(x_train, y_train))
print("Test set score: %f" % classifier.score(x_test, y_test))
print(classifier.n_layers_)
print(classifier.n_iter_)

{0: 11852, 1: 3781}
Test set has  1335 positives and  11615  negatives
Best classifier score
             precision    recall  f1-score   support

non-binding       0.91      0.77      0.84     11615
    binding       0.15      0.36      0.22      1335

avg / total       0.83      0.73      0.77     12950

Test set has  363 positives and  2320  negatives
Best classifier score
             precision    recall  f1-score   support

non-binding       0.89      0.77      0.82      2320
    binding       0.21      0.40      0.28       363

avg / total       0.80      0.72      0.75      2683

Test set has  1698 positives and  13935  negatives
Best classifier score
             precision    recall  f1-score   support

non-binding       0.91      0.77      0.84     13935
    binding       0.16      0.37      0.23      1698

avg / total       0.83      0.73      0.77     15633



Training set score: 0.746324
Test set score: 0.729099
3
124


In [53]:
#comparison to random

#prediction = classifier.predict(enzyme_x_test)
prediction = classifier.predict(data_rnd)
unique, counts = np.unique(prediction, return_counts=True)
print(dict(zip(unique, counts)))

classifierEstimation(classifier,enzyme_rnd,labels_enzyme_rnd)
classifierEstimation(classifier,dna_rnd,labels_dna_rnd)
classifierEstimation(classifier,data_rnd,labels_rnd)

print("Test set score: %f" % classifier.score(data_rnd,labels_rnd))

{0: 44187, 1: 33978}
Test set has  6581 positives and  58169  negatives
Best classifier score
             precision    recall  f1-score   support

non-binding       0.90      0.57      0.70     58169
    binding       0.11      0.46      0.17      6581

avg / total       0.82      0.56      0.64     64750

Test set has  1678 positives and  11737  negatives
Best classifier score
             precision    recall  f1-score   support

non-binding       0.88      0.57      0.69     11737
    binding       0.14      0.47      0.21      1678

avg / total       0.79      0.56      0.63     13415

Test set has  8259 positives and  69906  negatives
Best classifier score


             precision    recall  f1-score   support

non-binding       0.90      0.57      0.70     69906
    binding       0.11      0.46      0.18      8259

avg / total       0.82      0.56      0.64     78165

Test set score: 0.556643
